# Links

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import re
from itertools import zip_longest
import codecs
import json

headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:94.0) Gecko/20100101 Firefox/94.0', 'Accept-Language': 'en-US, en;q=0.5' }
 
links= ["https://www.jumia.com.eg/mobile-phones/?page=2#catalog-listing",
       "https://www.jumia.com.eg/tablets/?page=2#catalog-listing",
       "https://www.jumia.com.eg/computers-tablets/?page=2#catalog-listing",
       "https://www.jumia.com.eg/electronics-headphone/?page=2#catalog-listing"]   

product_name=[]      
product_link=[]      
product_ID=[] 
product_price=[]  
product_image=[]
product_ratings=[]   
product_revCounts=[] 
OldPrice=[]
Category=[]         
Sub_Category=[]  
product_spec=[]
product_det=[]
info_dic=[]
Seller_Name=[]
Seller_ID=[]
Seller_Link=[]
brand=[]

# Products

In [2]:
for url in links:
    flag= True
    while flag==True:
        print(url)
        response = requests.get(url, headers=headers)
        src=response.content
        soup = BeautifulSoup(response.content, 'lxml')

        products=soup.find_all("a",{"class":"core"})

        for i in range ( len(products) ):   
            # Product Name
            if products[i].find("h3", {"class":"name"}) != None:
                product_name.append( products[i].find("h3", {"class":"name"}).text) 
            else:
                pass

            # Product Link
            link= "https://www.jumia.com.eg/"+ (products[i].get('href'))
            product_link.append(link)

            # Product ID
            product_ID.append( products[i].get('data-id') )

            # Product Price
            product_price.append(products[i].find("div", {"class":"prc"}).text.replace("EGP", "").strip())

             # Product Image
            product_image.append(products[i].find('img').attrs['data-src'] ) 

            # Product Ratings    
            r= products[i].find("div", {"class":"stars _s"} ) 
            if products[i].find("div", {"class":"stars _s"} ) != None:
                product_ratings.append(r.text)
            else:
                product_ratings.append("")

            #Product Ratings Count    
            rc= products[i].find("div", {"class":"rev"} )
            if products[i].find("div", {"class":"stars _s"} ) != None:
                x= rc.text.split("(")
                y= x[1].split(")")
                product_revCounts.append(y[0])
            else:
                product_revCounts.append("")

            # Product Category and sub_category
            cat = products[i].get('data-category')
            co = cat.split("/", 2)
            Category.append(co[0])
            
            # Modify the subcategory condition
            if len(co) > 1:
                if co[1] == "Computers & Accessories":
                    Sub_Category.append("Laptops")
                else:
                    Sub_Category.append(co[1])
            else:
                Sub_Category.append("")

            # Product Sale
            if products[i].find("div",{"class":"s-prc-w"}) != None:
                OldPrice.append(products[i].find("div",{"class":"old"}).text.replace("EGP", "").strip())

            else:
                OldPrice.append("")
                
        while True:
            response = requests.get(url,headers=headers)
            soup = BeautifulSoup(response.content, 'lxml')
            pages = soup.find('div', {'class': 'pg-w -ptm -pbxl'})

            if(pages!=None):
                if pages.find("a", {"class":"pg", "aria-label":"Last Page"}):
                    url = "https://www.jumia.com.eg"+ str(pages.find("a", {"class":"pg", "aria-label":"Next Page"} ).get('href') )
                    break
                else:
                    #if false so it is the last page
                    flag=False
                    break   

https://www.jumia.com.eg/mobile-phones/?page=2#catalog-listing
https://www.jumia.com.eg/mobile-phones/?page=3#catalog-listing
https://www.jumia.com.eg/mobile-phones/?page=4#catalog-listing
https://www.jumia.com.eg/mobile-phones/?page=5#catalog-listing
https://www.jumia.com.eg/mobile-phones/?page=6#catalog-listing
https://www.jumia.com.eg/mobile-phones/?page=7#catalog-listing
https://www.jumia.com.eg/mobile-phones/?page=8#catalog-listing
https://www.jumia.com.eg/mobile-phones/?page=9#catalog-listing
https://www.jumia.com.eg/mobile-phones/?page=10#catalog-listing
https://www.jumia.com.eg/mobile-phones/?page=11#catalog-listing
https://www.jumia.com.eg/mobile-phones/?page=12#catalog-listing
https://www.jumia.com.eg/mobile-phones/?page=13#catalog-listing
https://www.jumia.com.eg/mobile-phones/?page=14#catalog-listing
https://www.jumia.com.eg/mobile-phones/?page=15#catalog-listing
https://www.jumia.com.eg/mobile-phones/?page=16#catalog-listing
https://www.jumia.com.eg/mobile-phones/?page=17#

In [4]:
len(product_name)

3927

# Products Details

In [6]:
for l in product_link:
    try:  
        result= requests.get(l)
        src= result.content
        soup= BeautifulSoup(src, 'lxml')

        # Product Brand
        div_elements = soup.find_all('div', {'class': '-pvxs'})
        brand_found = False
        for div in div_elements:
            if 'Brand:' in div.text:
                a_element = div.find('a')
                if a_element is not None and 'href' in a_element.attrs:
                    brand_name = a_element.text.upper()
                    brand.append(brand_name)
                    brand_found = True
                    break  
        if not brand_found:
            brand.append('')

        # Product Details
        dets = []
        section_tag = soup.find('div', {'class': 'markup -mhm -pvl -oxa -sc'})
        if section_tag != None:
            details = [detail.text.strip() for detail in section_tag.find_all('li')]
        product_det.append(" ".join(details))

         # Product Specifications
        specs = []
        section_tag = soup.find('section', {'class': 'card aim -mtm -fs16'})
        if section_tag != None:
            li_tags = section_tag.find_all('li')
            for li_tag in li_tags:
                li_text = li_tag.text.strip()
                specs.append(li_text)
            product_spec.append(specs)

        # Seller Name
        if soup.find("div", {"class":"-hr -pas"} ):
            seller_div = soup.find("div", {"class":"-hr -pas"})
            seller_name = seller_div.find("p", {"class":"-m -pbs"}).text.strip()
            Seller_Name.append(seller_name)
        else:
            Seller_Name.append("Jumia")

          # Seller Link And ID
        if soup.find("a", {"class":"-pas -df -i-ctr -upp"} ):
            s1= "https://www.jumia.com.eg/seller" + str(soup.find("a", {"class":"-pas -df -i-ctr -upp"} ).get('href') ) + "profile/"
            s1=  s1.replace('/ar/', '/')
            Seller_Link.append(s1)
            se1= s1.split("/seller",1)
            se2= se1[1].split("/",2)
            Seller_ID.append(se2[1])    
        else:
            pass
        
    except requests.exceptions.ConnectionError:
        print(f"ConnectionError occurred for link: {l}")
        continue

In [7]:
# Create CSV File
file_list= [product_name, Category, Sub_Category, brand, product_ID, product_price, product_image, product_link, product_ratings, product_revCounts, OldPrice, product_det, product_spec, Seller_Name, Seller_ID, Seller_Link]
exported= zip_longest(*file_list)

col= ["Product Name", "Category", "Sub Category", "Brand", "product ID", "Price", "Image", "Link", "Ratings", "Review Counts", "Old Price", "Details", "Specifications", "Seller", "Seller ID", "Seller Link"]
import pandas as pd
df= pd.DataFrame(exported,columns=col)

df.to_csv('C:\\Users\\aa\\Desktop\\Jumia_Products.csv')
df

,Product Name,Category,Sub Category,Brand,product ID,Price,Image,Link,Ratings,Review Counts,Old Price,Details,Specifications,Seller,Seller ID,Seller Link
0,"realme 9i, 128GB, 4GB, 6.6-inch, Dual-SIM, GB ...",Phones & Tablets,Mobile Phones,REALME,RE540MP0W3OJFNAFAMZ,"5,750.00",https://eg.jumia.is/unsafe/fit-in/300x300/filt...,https://www.jumia.com.eg//realme-9i-128gb-4gb-...,5 out of 5,1,"7,735.00",Chipset : Qualcomm Snapdragon 680Screen : 6.6-...,"[128 GB, 4GB Ram, 6.6-inch, Dual-SIM, Fast Cha...",uni-group.,yalla-tager,https://www.jumia.com.eg/seller/yalla-tager/pr...
1,Samsung Galaxy A04s - 6.5-inch 4GB/128GB Dual ...,Phones & Tablets,Mobile Phones,SAMSUNG,SA024MP11O7X7NAFAMZ,"5,050.00",https://eg.jumia.is/unsafe/fit-in/300x300/filt...,https://www.jumia.com.eg//galaxy-a04s-a-6.5-in...,3.8 out of 5,5,"6,666.00",NETWORK\tTechnology GSM / HSPA / LTE 2G bands\...,"[Display Size In Inches: 6.5 inches, Processor...",Dream2000 EG Marketplace,dream2000-eg-marketplace,https://www.jumia.com.eg/seller/dream2000-eg-m...
2,Samsung Galaxy A04s - 6.5-inch 4GB/128GB Dual ...,Phones & Tablets,Mobile Phones,SAMSUNG,SA024MP1JJ82JNAFAMZ,"5,050.00",https://eg.jumia.is/unsafe/fit-in/300x300/filt...,https://www.jumia.com.eg//galaxy-a04s-6.5-inch...,5 out of 5,1,"6,666.00",NETWORK\tTechnology GSM / HSPA / LTE 2G bands\...,"[Display Size In Inches: 6.5 inches, Processor...",Dream2000 EG Marketplace,dream2000-eg-marketplace,https://www.jumia.com.eg/seller/dream2000-eg-m...
3,Nokia C10 – 6.52 Inch – 32GB/2GB – Dual SIM 3G...,Phones & Tablets,Mobile Phones,NOKIA,NO228MP137FEDNAFAMZ,"2,499.00",https://eg.jumia.is/unsafe/fit-in/300x300/filt...,https://www.jumia.com.eg//nokia-c10-6.52-inch-...,,,"2,799.00",Technology: GSM / HSPA 2G bands: GSM 850 / 900...,"[6.52 Inch FHD + IPS LCD, 400 nits (typ), 32GB...",Jumia,,https://www.jumia.com.eg/sellerNoneprofile/
4,Samsung Galaxy A54 - 6.4 Inches - 8GB Ram -128...,Phones & Tablets,Mobile Phones,SAMSUNG,SA024MP16F764NAFAMZ,"14,777.00",https://eg.jumia.is/unsafe/fit-in/300x300/filt...,https://www.jumia.com.eg//samsung-galaxy-a54-6...,,,"15,999.00",Technology: GSM / HSPA / LTE / 5G SIM: Single ...,"[6.4 inches Super AMOLED, 128GB Storage, 8 GB ...",Jumia,,https://www.jumia.com.eg/sellerNoneprofile/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3922,Clear Bass Stereo In-Ear Earphones 3.5mm Wired...,Electronics,Headphones,,GE810EA0JF8CJNAFAMZ,377.00,https://eg.jumia.is/unsafe/fit-in/300x300/filt...,https://www.jumia.com.eg//generic-clear-bass-s...,,,678.00,,None,JG-Qiqi&Popo,None,None
3923,Clear Bass Stereo In-Ear Earphones 3.5mm Wired...,Electronics,Headphones,,GE810EA0ZOD5VNAFAMZ,377.00,https://eg.jumia.is/unsafe/fit-in/300x300/filt...,https://www.jumia.com.eg//generic-clear-bass-s...,,,678.00,,None,JG-Qiqi&Popo,None,None
3924,Soft Sleep Earphone Headphones 3.5mm Earphone ...,Electronics,Headphones,,GE810EA0PAVFQNAFAMZ,377.00,https://eg.jumia.is/unsafe/fit-in/300x300/filt...,https://www.jumia.com.eg//generic-soft-sleep-e...,,,678.00,,None,JG-Angan,None,None
3925,Soft Sleep Earphone Headphones 3.5mm Earphone ...,Electronics,Headphones,,GE810EA117JJANAFAMZ,377.00,https://eg.jumia.is/unsafe/fit-in/300x300/filt...,https://www.jumia.com.eg//generic-soft-sleep-e...,,,678.00,,None,JG-Angan,None,None


# Product Reviews

In [8]:
# Reviews
reviews_dic=[]
for i in product_ID:
    product_review=[]
    review_ratings=[]
    for iterations in range(1,40):
        R_url= f"https://www.jumia.com.eg/catalog/productratingsreviews/sku/{i}/?page={iterations}"
        result= requests.get(R_url, headers=headers)
        soup= BeautifulSoup(result.content, 'lxml')
        
        if soup.find("div", {"class":"cola -phm -df -d-co"} ) != None:
             # Reviews
            rev_block= soup.find_all("p", {"class":"-pvs"} )
            if(len(rev_block) != 0):
                for itr in range(len(rev_block)):
                    rev=rev_block[itr].text
                    # Rating Review
                    if soup.find("div", {"class":"stars _m _al -mvs"} ) !=None:
                        rev_rate= soup.find("div", {"class":"stars _m _al -mvs"} ).text
                        rate=rev_rate[0]
                        product_rev={
                            "ProductID":i,
                            "ProductReviews":rev,
                            "Review Rating":int(rate) }
                        reviews_dic.append(product_rev)
                        
                    else:
                        break
            else:
                break              
            
            arrows= soup.find("div", {"class":"pg-w -mhm -ptl -pbxl -bt"} )
            if(arrows!=None):
                if arrows.find("a", {"class":"pg", "aria-label":"Next Page"}):
                    pass
                else:
                    break
            else:
                break                      
        else:
            break

In [9]:
with codecs.open('Jumia_Products_Reviews.json','w','utf-8') as outfile:
    outfile.write(json.dumps(reviews_dic, ensure_ascii=False))

# Seller Reviews

In [10]:
# Seller Reviews
Unique_Seller_Name = set(Seller_Name)
s_reviews_dic=[]
for s in Unique_Seller_Name:
    seller_review=[]
    s_review_ratings=[]
    for iterations in range(1,20):
        s_url= f"https://www.jumia.com.eg/seller/{s}/profile/?page={iterations}"
        result= requests.get(s_url, headers=headers)
        soup= BeautifulSoup(result.content, 'lxml')
        
        if soup.find("div", {"class":"card-b col16 row"} ) != None:
            s_block= soup.find_all("p", {"class":"-pvs"} )
            if(len(s_block) != 0):
                for itr in range(len(s_block)):
                    s_rev= s_block[itr].text
                     # Rating Review
                    if soup.find("div", {"class":"stars _m _al -mts"} ) !=None:
                        s_rev_rate= soup.find("div", {"class":"stars _m _al -mts"} ).text
                        rate=s_rev_rate[0]
                        seller_rev={
                            "SellerName":s,
                            "SellerReviews":s_rev,
                            "Seller Review Rating":int(rate) }
                        s_reviews_dic.append(seller_rev)
                        
                    else:
                        break
            else:
                break
                
            arrows= soup.find("div", {"class":"pg-w col16 -ptxl -pbm"} )
            
            if(arrows!=None):
                if arrows.find("a", {"class":"pg", "aria-label":"Next Page"}):
                    pass
                else:
                    break
            else:
                break
                           
        else:
            break

In [11]:
with codecs.open('Jumia_Sellers_Reviews.json','w','utf-8') as outfile:
    outfile.write(json.dumps(s_reviews_dic, ensure_ascii=False))

# Final Structure for the Data

In [15]:
products = []

# Iterate over the collected data and create product dictionaries
for i in range(len(product_name)):
    product = {
        "Marketplace": "Jumia",
        "ProductTitle": product_name[i],
        "ProductBrand": brand[i],
        "ProductCategory": Category[i],
        "ProductDescription": product_det[i],  
        "ProductID": product_ID[i],
        "ProductImage": product_image[i],
        "ProductLink": product_link[i],
        "ProductOldPrice": OldPrice[i],
        "ProductPrice": product_price[i],
        "ProductRatingCount": product_revCounts[i],
        "ProductRatings": product_ratings[i],
        "ProductReviews": [],  
        "ProductSpecifications": product_spec[i] if i < len(product_spec) else [],
        "ProductSubCategory": Sub_Category[i],
        "SellerID": Seller_ID[i] if i < len(Seller_ID) else None,
        "SellerName": Seller_Name[i],
        "SellerUrl": Seller_Link[i] if i < len(Seller_Link) else None
    }
    products.append(product)

for review in reviews_dic:
    product_id = review["ProductID"]
    for product in products:
        if product["ProductID"] == product_id:
            product["ProductReviews"].append({
                "review": review["ProductReviews"],
                "rating": review["Review Rating"]
            })
            break

json_data = json.dumps(products, indent=4, ensure_ascii=False)
with open('data/Jumia_Data.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_data)